In [17]:
import  cv2
import tkinter
import numpy as np
from matplotlib import pyplot as plt


# Defina la función de mejora lineal, <x1, x2> es el intervalo de mejora
def linear_threshold(Image,h,w,x1,x2,z2,z1 = 0,z3 = 0):

    for i in range(0, h):
        for j in range(0, w):
            tmp = Image[i, j]
            if tmp < x1:
                Image[i, j] = z1
            elif tmp >= x1 and tmp < x2:
                Image[i, j] = z2
            else:
                Image[i, j] = z3

    return Image

#Función de cálculo de área de contorno
def areaCal(contour):

    area = 0
    for i in range(len(contour)):
        area += cv2.contourArea(contour[i])

    return area

if __name__== '_main_':
    #Leer imágenes
    img = cv2.imread('areaglacial.jpeg',0)
    #Configurar la máscara, enfocar la imagen procesada al sándwich
    mask = np.zeros(img.shape[:2],np.uint8)
    # Tamaño de la máscara
    mask[800:1100,100:2400] = 255
    masked_img = cv2.bitwise_and(img,img,mask = mask)
    # Histograma en escala de grises
    hist_mask = cv2.calcHist([img], [0], mask, [256], [0, 256])
    # Resalte el primer plano y mejore las características de la imagen. El umbral <26,200> se estima aproximadamente en función del histograma. Dado que el histograma es un solo pico (la imagen es un poco escoria), la solución aún no se conoce y debe modificarse.
    threshold_image = linear_threshold(masked_img,1920,2560,26,200,255)
    #Filtrado medio
    median1 = cv2.medianBlur(threshold_image,5)
    
    # Generar imagen
    plt.subplot(2,2,1),plt.imshow(img,'gray')
    plt.subplot(2,2,2),plt.imshow(threshold_image,'gray')                                              
    plt.subplot(2, 2, 3), plt.plot(hist_mask)
    plt.subplot(2, 2, 4), plt.imshow(median1, 'gray')
    plt.show()
    
    
    Image = np.zeros((1920, 2560), np.uint8)
    # cv2.findContours () tiene tres parámetros: 1. imagen de entrada, 2. contorno, 3. estructura tomográfica. Hay tres valores de retorno: el segundo valor de retorno es una lista de Python, que almacena todos los contornos, y cada contorno es una matriz Numpy que contiene las coordenadas del punto límite del objeto (x, y). Para los dos valores de retorno restantes, consulte el documento chino de openCV.
    image,contours,hierarchv = cv2.findContours(median1,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)   
    color = cv2.cvtColor(Image,cv2.COLOR_GRAY2BGR)
    #Generar el contorno en imagen
    Image = cv2.drawContours(color,contours,-1,(0,0,255),1)
    plt.imshow(Image)
    plt.show()
    #Obtener área de contorno e imprimir
    print(areaCal(contours))